In [13]:
import importtestsmodule
import json
import kafka_client
import utils
import test_da
import test_jms
import pprint

config_path = "../config.json"
cred_twitter_path = "../twitter.cred"
cred_reddit_path = "../reddit.cred"
pp=pprint.PrettyPrinter()

## Configuration and topics

### Config

`kafka_server` - set to `localhost:9094` (use `kafka:9092` if you run it inside a container)

### Topics

`sentiment` - sentiment analyser

### Credentials

Stored in a file `cred.json`. Some data acquirers require credentials.


In [ ]:
pp.pprint(utils.load_config(cred_twitter_path))
pp.pprint(utils.load_config(cred_reddit_path))

# Tests

## Test data acquirer - data acquisition

**Prerequisities**:
- The data acquirer listents at a topic `config['topics']['dataacquirer_update_topic']`
- Credentials in case of a twitter data acquirer or reddit data acquirer.

**Steps**

- Creates new job
- Waits for one message
- Checks that the message is in the a correct post
  - Success if true, fails otherwise
- Stops the running job

In [ ]:
print("Testing Twitter data acquirer")
test_da.start_test("../config.json",cred_twitter_path)

print("Testing Reddit data acquirer")
test_da.start_test("../config.json",cred_reddit_path)

## Test job service registration and replay

**Prerequisites**:

- Jms must run (on `localhost:5100` in debug or `localhost:6009` in docker both defined in `config[uris][jms_submit_uri]`)

**Steps**:

- Registering new components
- Submit job config
- Assert received job configs for each component
  - Asserting data acquirer job config
  - Asserting analyser job config
- Assert replayed data acquirer config
  - Asserting data acquirer job config
  - Asserting analyser job config

In [11]:
test_jms.start_test("../config.json")

config loaded successfully


NoBrokersAvailable: NoBrokersAvailable

## Test analysers

**Prerequisites**:

n/a

**Steps**:
 
TODO

In [49]:
client = kafka_client.KafkaClient("localhost:9094")


In [51]:
regpost = client.get_message_from_topic("job_management.registration.request")
print(regpost)

{'ComponentId': 'DataAnalyser_topics', 'ComponentType': 'DATA_ANALYSER', 'UpdateChannelName': 'job_management.job_configuration.DataAnalyser_topics', 'InputChannelName': 'job_management.component_data_input.DataAnalyser_topics', 'attributes': {'outputFormat': {'topics': 'textListValue'}}}


ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv6 ('::1', 9094, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network d

ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv6 ('::1', 9094, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network d

ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv6 ('::1', 9094, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094

ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv6 ('::1', 9094, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094

ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv6 ('::1', 9094, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094

ERROR:kafka.conn:<BrokerConnection node_id=1001 host=localhost:9094 <connected> [IPv4 ('127.0.0.1', 9094)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\kafka\conn.py", line 942, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


In [43]:
output_topic = "job_management.component_data_analyzed_input.storage_db"

topic = "job_management.component_data_input.DataAnalyser_topics"
client.produce_post(topic,text="this is totally random text")

post = client.get_message_from_topic(output_topic)
print(post)

KafkaTimeoutError: KafkaTimeoutError: Batch for TopicPartition(topic='job_management.component_data_input.DataAnalyser_topics', partition=0) containing 1 record(s) expired: 30 seconds have passed since batch creation plus linger time